# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns

# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
# 实例化类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
        host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
        user="ylc",
        password="1O8t5lcJ5aMhwwPEUUjS",
        database = '' ,
        port=3306
        ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips   
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, tor.update_time, tomt.reason, pa.type, om.order_type, tp.classify_id
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )               -- 近15天数据
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-06'
;
'''

df_order = query(sql1)
df_order.shape

(267289, 40)

In [5]:
sql_risk = ''' -- risk等级表
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risk = query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [8]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(267287)

# 处理日期

In [9]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["下单日期"].dt.strftime('%Y-%m')
df.shape

(267287, 43)

In [10]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 处理备注信息

In [11]:
df = df[df['sku_attributes'].notnull()]

In [12]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [13]:
df['免审'] = np.where((df.is_vip=='1')&(df.status_result=='0'),1,0)

# 归属渠道

In [14]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [15]:
df = clean.order_drop_duplicates(df)

# 定义状态

In [16]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

# 导入服务订单

In [17]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-09-26'

In [18]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(34783, 68)

In [19]:
# 获取各个节点的状态
df = clean.status_node(df)

# 剔除商家信息

In [20]:
# 保留商家信息
df_merchant = df.copy()

In [21]:
# 剔除商家数据
df = clean.drop_merchant(df)

# 导入出库台账数据

In [22]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")   #,skiprows=0 header 
df_ck.shape

(36102, 18)

In [23]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(2484, 101)

In [24]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

In [25]:
#按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)

# 剔除据量数据

In [26]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

In [27]:
df2.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'tips', 'is_vip', 'status_result',
       'qvt_risk', 'qvt_result', 'channel_name', 'channel_type_id',
       'activity_name', 'merchant_id', 'merchant_name',
       'total_freeze_fund_amount', 'buy_service_product', 'service_status',
       'order_method', 'update_time', 'reason', 'type', 'order_type',
       'classify_id', '下单日期', '月份', '下单月份', '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因',
       '商品类型', '租赁方案', '押金类型', '优惠券使用否', '是否进行预授权', '颜色', '内存', '进件', '来源渠道',
       '免审', '归属渠道', '状态编码', 'status_r', 'status_re', 'time_ra', 'status_ra',
       '是否前置拦截', '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '人审拒绝',
       '客户取消', '无法联系', '是否进件', '是否出库',

# 总体

In [28]:
#总体
df_group_weekday = all_models.data_group(df, df2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum = week_models.week_data_group(df_group_weekday, 4)
weekly_sum = weekly_sum[['week_group', '去重订单数', '去重订单数环比', '进件数', '进件数环比', '预授权通过率', '预授权率环比差值', '出库', '出库环比', '进件出库率', '进件出库率环比差值', '订单出库率', '订单出库率环比差值']]
weekly_sum.set_index('week_group')

,去重订单数,去重订单数环比,进件数,进件数环比,预授权通过率,预授权率环比差值,出库,出库环比,进件出库率,进件出库率环比差值,订单出库率,订单出库率环比差值
week_group,,,,,,,,,,,,
20250912-20250918,37296,nan%,10926,nan%,29.30%,nan%,858,nan%,7.85%,nan%,2.30%,nan%
20250919-20250925,39102,4.84%,11547,5.68%,29.53%,0.24%,807,-5.94%,6.99%,-0.86%,2.06%,-0.24%


# 搜索

In [29]:

df_ss = df[df["归属渠道"]=="搜索渠道"]
df_ss2 = df2[df2["归属渠道"]=="搜索渠道"]

df_group_weekday_ss = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum_ss = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_ss, 4)
# weekly_sum_ss = weekly_sum_ss[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('week_group')
weekly_sum_ss[['去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,订单占比,进件数,进件占比,出库,出库占比,进件出库转化率,订单出库率
去重订单数,,,,,,,
9777,26.21%,2799,25.62%,277,32.28%,9.90%,2.83%
11580,29.61%,3199,27.70%,286,35.44%,8.94%,2.47%


In [30]:
# 自然周
weekly_sum_ss7 = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_ss, 7)[['进件数', '出库']]

# 芝麻

In [31]:
# df_zm = df[df["归属渠道"]=="芝麻租物"]
# df_zm2 = df2[df2["归属渠道"]=="芝麻租物"]

# df_group_weekday_zm = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
# weekly_sum_zm = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_zm, 1)
# weekly_sum_zm = weekly_sum_zm[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('week_group')
# # weekly_sum_zm[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']]#.set_index('去重订单数')
# with pd.ExcelWriter('F:/需求/七月需求/上周二到本周一转化数据.xlsx', engine='xlsxwriter') as writer:
#     weekly_sum_zm.to_excel(writer, sheet_name='芝麻租物')
#     weekly_sum_ss.to_excel(writer, sheet_name='搜索渠道')

# 单人聊天

In [32]:
df_dr=df[df["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df_dr2=df2[df2["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]

df_group_weekday_dr = all_models.data_group(df_dr, df_dr2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]

# 芝麻租物

In [33]:
df_zm=df[df["归属渠道"]=="芝麻租物"]
df_zm2=df2[df2["归属渠道"]=="芝麻租物"]

df_group_weekday_zm = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum_zm = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_zm, 4)
# weekly_sum_zm[['week_group', '去重订单数', '订单占比', '是否进件', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')
weekly_sum_zm[['去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,订单占比,进件数,进件占比,出库,出库占比,进件出库转化率,订单出库率
去重订单数,,,,,,,
13683,36.69%,2293,20.99%,336,39.16%,14.65%,2.46%
13198,33.75%,2120,18.36%,258,31.97%,12.17%,1.95%


# 抖音

In [34]:
classify_id_list = [125,126,127,185]
df_dy=df[(df["归属渠道"]=="抖音渠道")&(df.classify_id.isin(classify_id_list))]
df_dy2=df2[(df2["归属渠道"]=="抖音渠道")&(df2.classify_id.isin(classify_id_list))]

df_group_weekday_dy = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
df_group_weekday_dy = clean.continuous_dates(df_group_weekday_dy).reset_index(names=['下单日期']).set_index('下单日期')

# 支付宝直播

In [35]:
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]

df_group_weekday_df_zfb = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]

# 总体剔除直播数据

In [36]:
weekly_sum_tc = df_group_weekday - df_group_weekday_dr - df_group_weekday_dy - df_group_weekday_df_zfb
weekly_sum_tc['出库'] =  df_group_weekday['出库'] - df_group_weekday_dr['出库'] - df_group_weekday_dy['出库'] - df_group_weekday_df_zfb['出库']
weekly_sum_tc_group = week_models.week_data_channel_group(df_group_weekday, weekly_sum_tc, 4)
# weekly_sum_tc_group[['week_group', '去重订单数', '是否进件', '出库', '预授权通过率', '进件出库转化率', '订单出库率']].set_index('去重订单数')
weekly_sum_tc_group[['去重订单数', '进件数', '出库', '预授权通过率', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,进件数,出库,预授权通过率,进件出库转化率,订单出库率
去重订单数,,,,,
34823.0,10239.0,846.0,29.40%,8.26%,2.43%
36745.0,10909.0,803.0,29.69%,7.36%,2.19%


# 免审

In [ ]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]

df_group_ms = all_models.data_group(df_ms, df_ms2, df_risk_examine, '下单日期')[['进件数', '出库']].rename(columns={'进件数': '免审进件', '出库': '免审出库'})
df_group_ms['总体进件'] = df_group_weekday['进件数'][:-1]
df_group_ms['芝麻租物进件'] = df_group_weekday_zm['进件数'][:-1]

# 各渠道每日免审出库单数统计
df_group_ms['总体出库'] = df_group_weekday['出库'][:-1]
df_group_ms['芝麻租物出库'] = df_group_weekday_zm['出库'][:-1]

# 免审出库
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]
dfdfck_ms = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True).reset_index()[['下单日期', '芝麻租物','搜索渠道']][:-1]
dfdfck_ms['下单日期'] = pd.to_datetime(dfdfck_ms['下单日期'])
dfdfck_ms.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
# 获取渠道免审进件
dfms = pd.crosstab(df_ms["下单日期"],df_ms["归属渠道"],margins=True).reset_index()[:-1]
dfms['下单日期']  = pd.to_datetime(dfms['下单日期'])
dfms = dfms[['下单日期', "芝麻租物","搜索渠道"]]

dfms.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索免审进件'}, inplace=True)
df_group_weekday_ms = pd.merge(df_group_ms, dfms, on='下单日期', how='outer')#[:-1]

# 关联数据
df_group_weekday_ms = pd.merge(df_group_weekday_ms, dfdfck_ms, on='下单日期', how='outer')
df_group_weekday_ms.set_index('下单日期', inplace=True)
# 进行周汇总
weekly_sum_ms = week_models.week_data_m_group(df_group_weekday_ms, 7, '免审')
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件']/weekly_sum_ss7['进件数'] * 100
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库']/weekly_sum_ss7['出库'] * 100
weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms[['week_group', '总体进件', '芝麻租物进件', '免审进件', '芝麻租物免审进件', '搜索免审进件', '免审进件占比', '芝麻租物免审进件占比', '搜索免审进件占比', '总体出库', '芝麻租物出库', '免审出库', '芝麻租物免审出库', '搜索免审出库', '免审转化率', '免审出库占比', '芝麻租物免审出库占比', '搜索免审出库占比']].set_index('week_group')

,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比
week_group,,,,,,,,,,,,,,,,,
20250908-20250914,11314.0,2593.0,2777,1194,826,24.54%,46.05%,31.45%,887.0,356.0,868,347.0,259.0,31.26%,97.86%,97.47%,99.62%
20250915-20250921,11607.0,2303.0,2772,970,1047,23.88%,42.12%,33.63%,837.0,302.0,823,290.0,285.0,29.69%,98.33%,96.03%,97.94%


# 免审订单转化

In [ ]:
df_msdd = df[(df.is_vip=='1')&(df.status_result=='0')]
df_msdd2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_group_mszh = all_models.data_group(df_msdd, df_msdd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]

weekly_sum_mszh = week_models.week_data_m_group(df_group_mszh, 7, '免审订单')
weekly_sum_mszh[['week_group', '进件数', '人审拒绝', '客户取消', '出库', '待审核', '进件出库率', '取消率', '人审拒绝率', '出库前风控强拒', '出库前强拒比例', '无法联系', '无法联系占比']].set_index('week_group')
# weekly_sum_mszh = weekly_sum_mszh[['week_group', "进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].set_index('week_group')
# weekly_sum_mszh

,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率,出库前风控强拒,出库前强拒比例,无法联系,无法联系占比
week_group,,,,,,,,,,,,
20250908-20250914,2777,1297.0,210,868,1,31.26%,7.56%,46.71%,51.0,1.84%,191,6.88%
20250915-20250921,2772,1285.0,175,823,130,29.69%,6.31%,46.36%,48.0,1.73%,181,6.53%


# 免押订单转化

## 预授权调起订单数

In [ ]:
# 基于All_Class.py中的Week_models模块的同名函数修改
def custom_weekly_resampler(df, key):
    '''
    W：默认表示每周六结束（即周五开始）。
    W-MON：表示每周一结束（即周二开始）。
    W-TUE：表示每周二结束（即周三开始）。
    W-WED：表示每周三结束（即周四开始）。
    W-THU：表示每周四结束（即周五开始）。
    W-FRI：表示每周五结束（即周六开始）。
    W-SAT：表示每周六结束（即周日开始）。
    W-SUN：表示每周日结束（即周一开始）。
    :param df: 传入的df，需要包含日期
    :return: 返回增加了按周的df
    '''
    # 确保日期列为datetime类型并设为索引
    df_copy = df.copy()
    df_copy['下单日期'] = pd.to_datetime(df_copy['下单日期'])
    df_copy.set_index('下单日期', inplace=True)
    
    # 初始化一个日期
    week_dates = None
    # 获取每周需要的日期
    if key == 1:
        week_dates = df_copy.index.to_period('W-MON').to_timestamp('W-MON')
    elif key == 2:
        week_dates = df_copy.index.to_period('W-TUE').to_timestamp('W-TUE')
    elif key == 3:
        week_dates = df_copy.index.to_period('W-WED').to_timestamp('W-WED')
    elif key == 4:
        week_dates = df_copy.index.to_period('W-THU').to_timestamp('W-THU')
    elif key == 5:
        week_dates = df_copy.index.to_period('W-FRI').to_timestamp('W-FRI')
    elif key == 6:
        week_dates = df_copy.index.to_period('W').to_timestamp('W')
    elif key == 7:
        week_dates = df_copy.index.to_period('W-SUN').to_timestamp('W-SUN')

    # 使用 asfreq 方法将数据重采样到需要的周数
    df_copy['week_group'] = week_dates
    # print("week_group:", df_copy)

    return df_copy


    # 按周进行分组 获取数据
# 按周进行分组 获取数据
def W_group(df, date_name='下单日期'):
    '''
    计算每周的数据
    :param df: 传入的带有自定义时间频率的df
    :return: 返回按周分组后的周数据
    '''
    # 获取每周的数据，排除不足七天的数据
    df['week_day'] = df['week_group'].value_counts()
    df = df.bfill(axis=0)
    df = df[df['week_day'] == 7]

    # 按每周进行分组并求和
    weekly_sum = df.groupby('week_group').sum()

    # 获取最小和最大日期 pd.to_datetime(weekly_sum['min_date']).dt.strftime('%m%d')
    df = df.reset_index()
    min_date = pd.to_datetime(df.groupby('week_group')[date_name].min()).dt.strftime('%Y%m%d')
    max_date = pd.to_datetime(df.groupby('week_group')[date_name].max()).dt.strftime('%Y%m%d')
    dates = min_date.astype(str) + '-' + max_date.astype(str)
    weekly_sum = weekly_sum.reset_index()
    weekly_sum['week_group'] = weekly_sum['week_group'].map(dates, na_action='ignore')
    # print('week_ly:',weekly_sum)

    return weekly_sum

In [ ]:
sql_ysq = '''
    select om.create_time, tmu.id_card_num,top.order_id,om.status
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    -- and top.total_freeze_fund_amount is not null
    -- and trade_no is not null
    '''
df_ysq = clean.query(sql_ysq)

In [ ]:
df_ysq_t = df_ysq.copy()
df_ysq_t["下单日期"]=df_ysq_t["create_time"].dt.date
df_ysq_t["下单日期"]=pd.to_datetime(df_ysq_t["下单日期"],errors="coerce")
df_ysq_t['是否前置拦截'] = np.where((df_ysq_t.result.str.contains('id_card不得为空') )|(df_ysq_t.result.str.contains('mobile校验不通过')) |(df_ysq_t.result.str.contains('name校验不通过'))
                        |(df_ysq_t.result.str.contains('年龄超过49岁或低于18岁'))|(df_ysq_t.result=='风控未通过')|(df_ysq_t.result=='黑名单用户'),1,0)
df_ysq_t.loc[:,"进件"]=np.where((df_ysq_t["status"]==1)|(df_ysq_t["status"]==13),"未进件","进件")
# 去除是否前置拦截=1且进件=未进件的行
df_ans = df_ysq_t[(df_ysq_t['是否前置拦截']==0)|(df_ysq_t['进件']=='进件')]
# 按下单日期与id_card_num去重
df_ans = df_ans.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# 按下单日期分组计算订单数
df_ans = df_ans.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权调起人数"}).reset_index()
# 日度数据转周度数据，并只保留完整（7天）的周度数据
# df_weekly = custom_weekly_resampler(df_ysq_t, 7)
week_sum_ysq = W_group(custom_weekly_resampler(df_ans, 7))
# week_sum_ysq

In [ ]:
sql_ysqmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount = 0
    and trade_no is not null
    and top.status in (2, 5)
    '''

df_ysqmy = clean.query(sql_ysqmy)
df_ysqmy.shape

(19569, 3)

In [ ]:
df_ysqmy_t = df_ysqmy.copy()
df_ysqmy_t["下单日期"]=df_ysqmy_t["create_time"].dt.date
df_ysqmy_t["下单日期"]=pd.to_datetime(df_ysqmy_t["下单日期"],errors="coerce")
df_ysqmy_t = df_ysqmy_t.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqmy_t = df_ysqmy_t.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "全免押"}).reset_index()
# 日度数据转周度数据，并只保留完整（7天）的周度数据
df_weekly = custom_weekly_resampler(df_ysqmy_t, 7)
week_sum_ysqmy = W_group(df_weekly)


In [ ]:
df_mydd1 = df[df["押金类型"]=="全免押"] 
df_mydd2 = df2[df2["押金类型"]=="全免押"]

df_group_mydd = all_models.data_group(df_mydd1, df_mydd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]
df_group_mydd['总进件'] = df_group_weekday['进件数']
myck = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)
df_group_mydd['出库'] = myck['全免押'][:-1]

weekly_sum_mydd = week_models.week_data_m_group(df_group_mydd, 7, '免审订单')
weekly_sum_mydd.insert(2, '免押进件占比', (week_sum_ysqmy.全免押/week_sum_ysq.预授权调起人数).map(lambda x: format(x, '.2%')))
# weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['进件数']/weekly_sum_mydd['总进件'] * 100
# weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_mydd[['week_group', '总进件', '免押进件占比', '进件数', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']].set_index('week_group')

,总进件,免押进件占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
week_group,,,,,,,,,,,,,,,,
20250908-20250914,11314,28.97%,10911,8016,73.47%,3031.0,1607,1424.0,13.05%,223,2,890.0,8.16%,2.04%,29.36%,14.73%
20250915-20250921,11607,28.36%,11088,8116,73.20%,3144.0,1639,1505.0,13.57%,213,138,828.0,7.47%,1.92%,26.34%,14.78%


# 非免押订单转化

In [ ]:
sql_ysqfmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day)
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount > 0
    and trade_no is not null
    and top.status in (2, 5)
    '''
    
df_ysqfmy = clean.query(sql_ysqfmy)

df_ysqfmy["下单日期"]=df_ysqfmy["create_time"].dt.date
df_ysqfmy["下单日期"]=pd.to_datetime(df_ysqfmy["下单日期"],errors="coerce")
df_ysqfmy = df_ysqfmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqfmy = df_ysqfmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "部分免押"}).reset_index()
week_sum_ysqfmy = W_group(custom_weekly_resampler(df_ysqfmy, 7))
# week_sum_ysqfmy

In [ ]:
df_fmydd1 = df[df["押金类型"]=="部分免押"]
df_fmydd2 = df2[df2["押金类型"]=="部分免押"]

df_group_fmydd = all_models.data_group(df_fmydd1, df_fmydd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]
df_group_fmydd['总进件'] = df_group_weekday['进件数']
weekly_sum_fmydd = week_models.week_data_m_group(df_group_fmydd, 7, '免审订单')
weekly_sum_fmydd.insert(2, '非免押进件占比', (week_sum_ysqfmy.部分免押/week_sum_ysq.预授权调起人数).map(lambda x: format(x, '.2%')))
# weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['进件数']/weekly_sum_fmydd['总进件'] * 100
# weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['非免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_fmydd[['week_group', '总进件', '非免押进件占比', '进件数', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']].set_index('week_group')

,总进件,非免押进件占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
week_group,,,,,,,,,,,,,,,,
20250908-20250914,11314,1.91%,403,397.0,98.51%,7.0,5.0,2.0,0.50%,0,0,1,0.25%,0.00%,14.29%,1.24%
20250915-20250921,11607,2.31%,519,507.0,97.69%,12.0,9.0,3.0,0.58%,0,0,0,0.00%,0.00%,0.00%,1.73%


# 商家周数据

## 澄心优租

In [ ]:
week_cxyz = week_models.week_data_group_merchant(df_merchant, '澄心优租', '下单日期', 7)
week_cxyz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]].set_index('week_group')

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,人审拒绝率,待审核,出库,进件出库率,订单出库率
week_group,,,,,,,,,,,,
20250908-20250914,14.0,0.0,0.00%,4.0,28.57%,4.0,4.0,100.00%,0.0,0.0,0.00%,0.00%
20250915-20250921,14.0,0.0,0.00%,5.0,35.71%,5.0,3.0,60.00%,1.0,0.0,0.00%,0.00%


## 优优2店

In [ ]:
# week_yy2 = week_models.week_data_group_merchant(df_merchant, '优优2店', '下单日期', 7, '商家')
# week_yy2[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 北京海鸟窝科技有限公司

In [ ]:
week_hnw = week_models.week_data_group_merchant(df_merchant, '北京海鸟窝科技有限公司', '下单日期', 7)
week_hnw[['week_group',"去重订单数","进件数","预授权通过率","出库","进件出库率"]].set_index('week_group')


,去重订单数,进件数,预授权通过率,出库,进件出库率
week_group,,,,,
20250908-20250914,5.0,2.0,40.00%,0.0,0.00%
20250915-20250921,8.0,3.0,37.50%,0.0,0.00%


## 租着用电脑数码

In [ ]:
week_zzy = week_models.week_data_group_merchant(df_merchant, '租着用电脑数码', '下单日期', 7)
week_zzy[['week_group',"去重订单数","进件数","预授权通过率","出库","进件出库率"]].set_index('week_group')


,去重订单数,进件数,预授权通过率,出库,进件出库率
week_group,,,,,
20250908-20250914,58,9,15.52%,1,11.11%
20250915-20250921,50,9,18.00%,1,11.11%


## 趣智数码

In [ ]:
week_zqsm = week_models.week_data_group_merchant(df_merchant, '趣智数码', '下单日期', 7)
week_zqsm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250908-20250914,18.0,0.0,0.00%,5.0,27.78%,5.0,1.0,0.0,0.00%,0.0,3.0,60.00%,20.00%,16.67%
20250915-20250921,15.0,0.0,0.00%,11.0,73.33%,11.0,0.0,4.0,36.36%,4.0,0.0,0.00%,0.00%,0.00%


## 汇客好租

In [ ]:
week_hkhz = week_models.week_data_group_merchant(df_merchant, '汇客好租', '下单日期', 7)
week_hkhz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250908-20250914,288,32,11.11%,131,45.49%,131,0,99,75.57%,0,4,3.05%,0.00%,1.39%
20250915-20250921,65,5,7.69%,25,38.46%,25,0,14,56.00%,0,2,8.00%,0.00%,3.08%


## 小蚂蚁租机

In [ ]:
# week_xmy = week_models.week_data_group_merchant(df_merchant, '小蚂蚁租机', '下单日期', 7)
# week_xmy[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 乙辉数码

In [ ]:
# week_yhsm = week_models.week_data_group_merchant(df_merchant, '乙辉数码', '下单日期', 7)
# week_yhsm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 兴鑫兴通讯

In [ ]:
# week_xxx = week_models.week_data_group_merchant(df_merchant, '兴鑫兴通讯', '下单日期', 7)
# week_xxx[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 瓜子笔记本电脑

In [ ]:
# week_gz = week_models.week_data_group_merchant(df_merchant, '呱子笔记本电脑', '下单日期', 7)
# week_gz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 南京聚格网络科技

In [ ]:
# week_jg = week_models.week_data_group_merchant(df_merchant, '南京聚格网络科技', '下单日期', 7)
# week_jg[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 人人享租

In [ ]:
week_rrxz = week_models.week_data_group_merchant(df_merchant, '人人享租', '下单日期', 7)
week_rrxz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250908-20250914,390,0,0.00%,389,99.74%,28,0,20,5.14%,0,7,1.80%,0.00%,1.79%
20250915-20250921,632,0,0.00%,631,99.84%,68,0,45,7.13%,3,18,2.85%,0.00%,2.85%


## 崇胜数码

In [ ]:
# week_cssm = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '崇胜数码', '下单日期', 7)
# week_cssm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 喜卓林租机

In [ ]:
# week_czl = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '喜卓灵租机', '下单日期', 7)
# week_czl[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 喜卓灵新租机

In [ ]:
week_xzlx = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '喜卓灵新租机', '下单日期', 7)
week_xzlx[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250908-20250914,1254,0,0.00%,1254,100.00%,94,0,64,5.10%,0,28,2.23%,0.00%,2.23%
20250915-20250921,916,0,0.00%,911,99.45%,111,1,76,8.34%,5,26,2.85%,0.11%,2.84%
